### Configurations

In [1]:
GENERATE_VIDEO = 0
TRACK_TEAMS = 1

### Imports

In [2]:
from inference import get_model
from ultralytics import YOLO
import supervision as sv
from tqdm import tqdm
import pandas as pd
import numpy as np
from sports.common.team import TeamClassifier
from sports.annotators.soccer import draw_pitch, draw_points_on_pitch
from sports.configs.soccer import SoccerPitchConfiguration
from sports.common.view import ViewTransformer
from supervision.draw.utils import draw_image
from typing import List, Union
from collections import deque

[04/15/25 22:21:01] WARNING  Your inference package version 0.46.4 is out of date! Please upgrade to ]8;id=44854;file:///Users/johncomonitski/anaconda3/envs/soccer_vision/lib/python3.11/site-packages/inference/core/__init__.py\__init__.py]8;;\:]8;id=929892;file:///Users/johncomonitski/anaconda3/envs/soccer_vision/lib/python3.11/site-packages/inference/core/__init__.py#41\41]8;;\
                             version 0.46.5 of inference for the latest features and bug fixes by                  
                             running `pip install --upgrade inference`.                                            

ModelDependencyMissing: Your `inference` configuration does not support PaliGemma model. Use pip install 'inference[transformers]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support Florence2 model. Use pip install 'inference[transformers]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support Qwen2.5-VL model. Use pip install 'inference[transformers]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[clip]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does

In [3]:
from dotenv import load_dotenv
import os

load_dotenv("./../env/keys.env")

# Access the variables
ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API")

In [4]:
os.environ["ONNXRUNTIME_EXECUTION_PROVIDERS"] = "[CUDAExecutionProvider]"

### Getting Your Model

You can either use Roboflow's pretrained models or repalce `PLAYER_DETECTION_MODEL_ID` and `FIELD_DETECTION_MODEL_ID` the Roboflow ID's of your customily trained models

In [5]:
PLAYER_DETECTION_MODEL_ID = "football-players-detection-3zvbc/11"
PLAYER_DETECTION_MODEL = get_model(model_id=PLAYER_DETECTION_MODEL_ID, api_key=ROBOFLOW_API_KEY)

In [6]:
FIELD_DETECTION_MODEL_ID = "football-field-detection-f07vi/14"
FIELD_DETECTION_MODEL = get_model(model_id=FIELD_DETECTION_MODEL_ID, api_key=ROBOFLOW_API_KEY)

### Data Source

Update `VIDEO_FILE` with the match footage you want to track

In [20]:
VIDEO_FILE = "2e57b9_0"
SOURCE_VIDEO_PATH = './footage/' + VIDEO_FILE + ".mp4"

---

**Classes**

In [8]:
objects = {
    "ball" : 0,
    "goalkeeper" : 1,
    "player" : 2,
    "referee" :3
}

**Annotations**

In [10]:
ellipse_annotator = sv.EllipseAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000'),
    text_position=sv.Position.BOTTOM_CENTER
)
triangle_annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FFD700'),
    base=25,
    height=21,
    outline_thickness=1
)
box_annotator = sv.BoxAnnotator(
    color=sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)

**Identify Goal Keeper**

In [11]:
def resolve_goalkeepers_team_id(players, goalkeepers):
    goalkeepers_xy = goalkeepers.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    players_xy = players.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    team_0_centroid = players_xy[players.class_id == 0].mean(axis=0)
    team_1_centroid = players_xy[players.class_id == 1].mean(axis=0)
    goalkeepers_team_id = []
    for goalkeeper_xy in goalkeepers_xy:
        dist_0 = np.linalg.norm(goalkeeper_xy - team_0_centroid)
        dist_1 = np.linalg.norm(goalkeeper_xy - team_1_centroid)
        goalkeepers_team_id.append(0 if dist_0 < dist_1 else 1)

    return np.array(goalkeepers_team_id)

**Get Detections**

In [12]:
def get_detections(frame, detections, key_points, tracker, team_classifier):
  CONFIG = SoccerPitchConfiguration()

  # Organize Detections
  ball_detections = detections[detections.class_id == objects["ball"]]
  ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)

  all_detections = detections[detections.class_id != objects["ball"]]
  all_detections = all_detections.with_nms(threshold=0.5, class_agnostic=True)
  all_detections = tracker.update_with_detections(detections=all_detections)

  goalkeepers_detections = all_detections[all_detections.class_id == objects["goalkeeper"]]
  players_detections = all_detections[all_detections.class_id == objects["player"]]

  if(team_classifier):
    players_crops = [sv.crop_image(frame, xyxy) for xyxy in players_detections.xyxy]
    players_detections.class_id = team_classifier.predict(players_crops)

  goalkeepers_detections.class_id = resolve_goalkeepers_team_id(
      players_detections, goalkeepers_detections)

  # Adjust Points to 2D Pitch
  filter = key_points.confidence[0] > 0.5
  frame_reference_points = key_points.xy[0][filter]
  pitch_reference_points = np.array(CONFIG.vertices)[filter]

  transformer = ViewTransformer(
      source=frame_reference_points,
      target=pitch_reference_points
  )

  frame_ball_xy = ball_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
  pitch_ball_xy = transformer.transform_points(points=frame_ball_xy)
  ball_detections.data["pitch_xy"] = pitch_ball_xy

  frame_goalkeepers_xy = goalkeepers_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
  pitch_goalkeepers_xy = transformer.transform_points(points=frame_goalkeepers_xy)
  goalkeepers_detections.data["pitch_xy"] = pitch_goalkeepers_xy

  frame_players_xy = players_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
  pitch_players_xy = transformer.transform_points(points=frame_players_xy)
  players_detections.data["pitch_xy"] = pitch_players_xy

  # Merge Detections
  all_detections = sv.Detections.merge([ players_detections, goalkeepers_detections ])

  return (all_detections, ball_detections)

**Identify Teams**

In [17]:
def generate_team_model(video, PLAYER_DETECTION_MODEL):
  STRIDE = 30

  frame_generator = sv.get_video_frames_generator(
      source_path=SOURCE_VIDEO_PATH, stride=STRIDE)

  crops = []
  for frame in tqdm(frame_generator, desc='collecting crops'):
      result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
      detections = sv.Detections.from_inference(result)
      players_crops = [sv.crop_image(frame, xyxy) for xyxy in detections.xyxy]
      crops += players_crops

  #team_classifier = TeamClassifier(device="cuda")
  team_classifier = TeamClassifier()
  team_classifier.fit(crops)

  return team_classifier

**Output Results**

In [14]:
def save_tracking_results(players, ball, frames):
  csv = "Frame,Object,Object ID,Team,X1,Y1,X1,X2,X_Pitch,Y_Pitch\n"
  for frame in range(1, frames):
    for player in players:
      player_data = players[player]
      if str(frame) in player_data:
        csv += str(frame) + ",player," + str(player) + "," + str(player_data[str(frame)]["Team"]) + "," + str(player_data[str(frame)]["X1"]) + "," + str(player_data[str(frame)]["Y1"]) + ","  + str(player_data[str(frame)]["X2"]) + "," + str(player_data[str(frame)]["Y2"]) + "," + str(player_data[str(frame)]["X_Pitch"]) + "," + str(player_data[str(frame)]["Y_Pitch"]) + "\n"
    if str(frame) in ball:
      csv += str(frame) + ",ball,,," + str(ball[str(frame)]["X1"]) + "," + str(ball[str(frame)]["Y1"]) + "," + str(ball[str(frame)]["X2"]) + "," + str(ball[str(frame)]["Y2"]) + "," + str(ball[str(frame)]["X_Pitch"]) + "," + str(ball[str(frame)]["Y_Pitch"]) + "\n"

  with open("./output/" + VIDEO_FILE + ".csv", "w") as file:
    file.write(csv)

**Tracking**

In [15]:
CONFIG = SoccerPitchConfiguration()
tracking_results = ""

#Get Video
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

tracker = sv.ByteTrack()
tracker.reset()

In [18]:
team_classifier = None
if(TRACK_TEAMS):
  team_classifier = generate_team_model(SOURCE_VIDEO_PATH, PLAYER_DETECTION_MODEL)

collecting crops: 25it [00:17,  1.44it/s]
Embedding extraction: 19it [01:41,  5.35s/it]
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [19]:
#Iterate Over Each Frame
frame_number = 1
video_info = sv.VideoInfo.from_video_path(video_path=SOURCE_VIDEO_PATH)
players = {}
ball = {}

with sv.VideoSink(target_path="./output.mp4", video_info=video_info) as sink:
  for frame in tqdm(frame_generator, desc='Collecting Tracking Data...'):
    result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
    detections = sv.Detections.from_inference(result)

    result = FIELD_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
    key_points = sv.KeyPoints.from_inference(result)

    #Organize Detections
    all_detections, ball_detections = get_detections(frame, detections, key_points, tracker, team_classifier)
    object_ids = all_detections.tracker_id
    team_ids = all_detections.class_id
    object_types = all_detections.data["class_name"]
    pitch_xys = all_detections.data["pitch_xy"]
    ball_pitch_xys = ball_detections.data["pitch_xy"]
    all_detections.class_id = all_detections.class_id.astype(int)

    labels = [
        f"#{tracker_id}"
        for tracker_id
        in all_detections.tracker_id
    ]

    #Iterate Over Frames
    for idx, xyxy in enumerate(all_detections.xyxy):
      team_id = 0
      if(tracker):
        team_id = team_ids[idx]

      object_id = str(object_ids[idx])
      if(object_id not in players):
        players[object_id] = {}

      players[object_id][str(frame_number)] = {
          "Object Type" : object_types[idx],
          "Team" : team_id,
          "X1" : xyxy[0],
          "Y1" : xyxy[1],
          "X2" : xyxy[2],
          "Y2" : xyxy[3],
          "X_Pitch" : pitch_xys[idx][0],
          "Y_Pitch" : pitch_xys[idx][1],
          "Y_MPLSoccer" : float(float(pitch_xys[idx][1]) / float(CONFIG.width)),
          "X_MPLSoccer" : float(float(pitch_xys[idx][0]) / float(CONFIG.length))
      }

    if(ball_detections.xyxy.shape[0]):
      ball[str(frame_number)] = {
            "X1" : ball_detections.xyxy[0][0],
            "Y1" : ball_detections.xyxy[0][1],
            "X2" : ball_detections.xyxy[0][2],
            "Y2" : ball_detections.xyxy[0][3],
            "X_Pitch" : ball_pitch_xys[0][0],
            "Y_Pitch" : ball_pitch_xys[0][1],
            "Y_MPLSoccer" : float(float(ball_pitch_xys[0][1]) / float(CONFIG.width)),
            "X_MPLSoccer" : float(float(ball_pitch_xys[0][0]) / float(CONFIG.length))
      }
    else:
      ball[str(frame_number)] = {
            "X1" : 0,
            "Y1" : 0,
            "X2" : 0,
            "Y2" : 0,
            "X_Pitch" : 0,
            "Y_Pitch" : 0,
            "Y_MPLSoccer" : 0,
            "X_MPLSoccer" : 0
      }

    frame_number += 1

    if(GENERATE_VIDEO):
      annotated_frame = frame.copy()
      annotated_frame = ellipse_annotator.annotate(
          scene=annotated_frame,
          detections=all_detections)
      annotated_frame = label_annotator.annotate(
          scene=annotated_frame,
          detections=all_detections,
          labels=labels)
      annotated_frame = triangle_annotator.annotate(
          scene=annotated_frame,
          detections=ball_detections)

      sink.write_frame(frame=annotated_frame)

Embedding extraction: 1it [00:02,  2.95s/it]/s]
Embedding extraction: 1it [00:01,  1.63s/it]25s/it]
Embedding extraction: 1it [00:01,  1.54s/it]39s/it]
Embedding extraction: 1it [00:01,  1.60s/it]46s/it]
Embedding extraction: 1it [00:01,  1.58s/it]96s/it]
Embedding extraction: 1it [00:02,  2.78s/it]67s/it]
Embedding extraction: 1it [00:01,  1.80s/it]94s/it]
Embedding extraction: 1it [00:02,  2.11s/it]81s/it]
Embedding extraction: 1it [00:02,  2.71s/it]82s/it]
Embedding extraction: 1it [00:01,  1.67s/it]09s/it]
Embedding extraction: 1it [00:01,  1.74s/it].87s/it]
Embedding extraction: 1it [00:01,  1.68s/it].80s/it]
Embedding extraction: 1it [00:02,  2.05s/it].67s/it]
Embedding extraction: 1it [00:02,  2.79s/it].72s/it]
Embedding extraction: 1it [00:02,  2.81s/it].32s/it]
Embedding extraction: 1it [00:02,  2.56s/it].42s/it]
Embedding extraction: 1it [00:02,  2.06s/it].40s/it]
Embedding extraction: 1it [00:01,  1.86s/it].23s/it]
Embedding extraction: 1it [00:02,  2.08s/it].06s/it]
Embeddi

In [21]:
save_tracking_results(players, ball, frame_number)